## Introducing Distributed Word Vectors
This part of the tutorial will focus on using distributed word vectors created by the Word2Vec algorithm. (For an overview of deep learning, as well as pointers to some additional tutorials, see the "What is Deep Learning?" page).

Parts 2 and 3 assume more familiarity with Python than Part 1. We developed the following code on a dual-core Macbook Pro, however, we have not yet run the code successfully on Windows. If you are a Windows user and you get it working, please leave a note on how you did it in the forum! For more detail, see the "Setting Up Your System" page.

Word2vec, published by Google in 2013, is a neural network implementation that learns distributed representations for words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

Word2Vec does not need labels in order to create meaningful representations. This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics. Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math. The famous example is that, with highly trained word vectors, "king - man + woman = queen."

Check out Google's code, writeup, and the accompanying papers. This presentation is also helpful. The original code is in C, but it has since been ported to other languages, including Python. We encourage you to play with the original C tool, but be warned that it is not user-friendly if you are a beginning programmer (we had to manually edit the header files to compile it).

Recent work out of Stanford has also applied deep learning to sentiment analysis; their code is available in Java. However, their approach, which relies on sentence parsing, cannot be applied in a straightforward way to paragraphs of arbitrary length.

Distributed word vectors are powerful and can be used for many applications, particularly word prediction and translation. Here, we will try to apply them to sentiment analysis.



In [35]:
import pandas as pd

# Read data from files 
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print(train.shape)
print(test.shape)
print(unlabeled_train.shape)

(25000, 3)
(25000, 2)
(50000, 2)


In [36]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [37]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [38]:
unlabeled_train.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [41]:
# Import various modules for string cleaning

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
      
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 5. Return a list of words
    return(words)


# """"""""""""""
# # What this does?
# # 1. 
# # 2. 
# # 3. 
# # 4. 
# """"""""""""""


Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.



In [42]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


In [44]:
sentences = []  # Initialize an empty list of sentences

print( "Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/keonpark/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabr

Parsing sentences from unlabeled set


/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/keonpark/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP c

In [45]:
# Check how many sentences we have in total
len(sentences)

795538

In [46]:
sentences[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

### Training and Saving Your Model
With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

* Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.


* Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.


* Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.


* Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).


* Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.


* Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.


Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [47]:
# !pip install --upgrade gensim

In [48]:
import gensim


In [49]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters

num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

# If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context_trial1"
model.save(model_name)

2018-04-20 15:57:47,407 : INFO : collecting all words and their counts
2018-04-20 15:57:47,411 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-20 15:57:47,529 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training model...


2018-04-20 15:57:47,617 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-04-20 15:57:47,704 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-04-20 15:57:47,775 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-04-20 15:57:47,847 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-04-20 15:57:47,927 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-04-20 15:57:47,991 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-04-20 15:57:48,078 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-04-20 15:57:48,168 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-04-20 15:57:48,254 : INFO : PROGRESS: at sentence #100000, processed 2226967 words, keeping 50

2018-04-20 15:57:53,478 : INFO : PROGRESS: at sentence #740000, processed 16553079 words, keeping 119668 word types
2018-04-20 15:57:53,571 : INFO : PROGRESS: at sentence #750000, processed 16771406 words, keeping 120295 word types
2018-04-20 15:57:53,638 : INFO : PROGRESS: at sentence #760000, processed 16990810 words, keeping 120930 word types
2018-04-20 15:57:53,729 : INFO : PROGRESS: at sentence #770000, processed 17217947 words, keeping 121703 word types
2018-04-20 15:57:53,859 : INFO : PROGRESS: at sentence #780000, processed 17448093 words, keeping 122402 word types
2018-04-20 15:57:53,974 : INFO : PROGRESS: at sentence #790000, processed 17675169 words, keeping 123066 word types
2018-04-20 15:57:54,039 : INFO : collected 123504 word types from a corpus of 17798270 raw words and 795538 sentences
2018-04-20 15:57:54,040 : INFO : Loading a fresh vocabulary
2018-04-20 15:57:54,149 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-04-20 15:

2018-04-20 15:58:45,942 : INFO : EPOCH 3 - PROGRESS: at 23.38% examples, 585442 words/s, in_qsize 7, out_qsize 0
2018-04-20 15:58:46,970 : INFO : EPOCH 3 - PROGRESS: at 27.74% examples, 578162 words/s, in_qsize 6, out_qsize 1
2018-04-20 15:58:47,971 : INFO : EPOCH 3 - PROGRESS: at 32.34% examples, 578090 words/s, in_qsize 8, out_qsize 0
2018-04-20 15:58:48,985 : INFO : EPOCH 3 - PROGRESS: at 37.06% examples, 579801 words/s, in_qsize 6, out_qsize 1
2018-04-20 15:58:49,987 : INFO : EPOCH 3 - PROGRESS: at 41.98% examples, 585143 words/s, in_qsize 7, out_qsize 0
2018-04-20 15:58:50,988 : INFO : EPOCH 3 - PROGRESS: at 47.10% examples, 592245 words/s, in_qsize 8, out_qsize 0
2018-04-20 15:58:51,997 : INFO : EPOCH 3 - PROGRESS: at 52.32% examples, 598311 words/s, in_qsize 7, out_qsize 0
2018-04-20 15:58:53,012 : INFO : EPOCH 3 - PROGRESS: at 57.57% examples, 604230 words/s, in_qsize 7, out_qsize 0
2018-04-20 15:58:54,022 : INFO : EPOCH 3 - PROGRESS: at 62.37% examples, 604603 words/s, in_qsiz

2018-04-20 15:59:48,970 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-20 15:59:48,971 : INFO : EPOCH - 5 : training on 17798270 raw words (12749310 effective words) took 22.6s, 564527 effective words/s
2018-04-20 15:59:48,973 : INFO : training on a 88991350 raw words (63748496 effective words) took 114.4s, 557468 effective words/s
2018-04-20 15:59:48,975 : INFO : precomputing L2-norms of word weight vectors
2018-04-20 15:59:49,087 : INFO : saving Word2Vec object under 300features_40minwords_10context_trial1, separately None
2018-04-20 15:59:49,090 : INFO : not storing attribute vectors_norm
2018-04-20 15:59:49,093 : INFO : not storing attribute cum_table
2018-04-20 15:59:49,640 : INFO : saved 300features_40minwords_10context_trial1


### Exploring the Model Results

In [57]:
# The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:

model.doesnt_match("man tiger woman child ".split())


/Users/keonpark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


'tiger'

In [58]:
model.doesnt_match("paris berlin london austria".split())


/Users/keonpark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [59]:
model.doesnt_match("france england germany berlin".split())


/Users/keonpark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [60]:
# We can also use the "most_similar" function to get insight into the model's word clusters:
model.most_similar("man")


/Users/keonpark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('woman', 0.6277197599411011),
 ('lady', 0.5923646688461304),
 ('lad', 0.5720973610877991),
 ('monk', 0.5439918637275696),
 ('farmer', 0.5352869629859924),
 ('soldier', 0.5302597284317017),
 ('millionaire', 0.5279288291931152),
 ('men', 0.5108588933944702),
 ('sailor', 0.510452926158905),
 ('guy', 0.5053010582923889)]

In [61]:
model.most_similar("awful")

/Users/keonpark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7869938611984253),
 ('atrocious', 0.7482832670211792),
 ('horrible', 0.7440119981765747),
 ('abysmal', 0.7227820754051208),
 ('dreadful', 0.6959949731826782),
 ('appalling', 0.6898812651634216),
 ('horrendous', 0.6722686886787415),
 ('horrid', 0.6460875272750854),
 ('laughable', 0.6188846230506897),
 ('lousy', 0.6172640323638916)]